In [ ]:
!pip install sentencepiece
!git clone https://github.com/google-research/bert


## Add Bertbase (bertsrc in kaggle)

In [ ]:
!pip uninstall tensorflow==2.4.1 --yes

!pip install tensorflow==1.15.0

In [ ]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf

import sentencepiece as spm

# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s :  %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

# Create Vocab file from Dataset

In [ ]:
regex_tokenizer = nltk.RegexpTokenizer("\w+")

def normalize_text(text):
  # lowercase text
  text = str(text).lower()
  # remove non-UTF
  text = text.encode("utf-8", "ignore").decode()
  # remove punktuation symbols
  text = " ".join(regex_tokenizer.tokenize(text))
  return text

def count_lines(filename):
  count = 0
  with open(filename) as fi:
    for line in fi:
      count += 1
  return count

In [ ]:
from tensorflow.keras.utils import Progbar
RAW_DATA_FPATH = "../input/51k-pretrain-dataset/51k.txt" #@param {type: "string"}
PRC_DATA_FPATH = "proc_dataset.txt" #@param {type: "string"}

# apply normalization to the dataset
# this will take a minute or two

total_lines = count_lines(RAW_DATA_FPATH)
bar = Progbar(total_lines)

with open(RAW_DATA_FPATH,encoding="utf-8") as fi:
  with open(PRC_DATA_FPATH, "w",encoding="utf-8") as fo:
    for l in fi:
      fo.write(normalize_text(l)+"\n")
      bar.add(1)

In [ ]:
MODEL_PREFIX = "tokenizer" #@param {type: "string"}
VOC_SIZE = 1987 #@param {type:"integer"}
SUBSAMPLE_SIZE = 12800000 #@param {type:"integer"}
NUM_PLACEHOLDERS = 256 #@param {type:"integer"}

SPM_COMMAND = ('--input={} --model_prefix={} '
               '--vocab_size={} --input_sentence_size={} '
               '--shuffle_input_sentence=true ' 
               '--bos_id=-1 --eos_id=-1').format(
               PRC_DATA_FPATH, MODEL_PREFIX, 
               VOC_SIZE - NUM_PLACEHOLDERS, SUBSAMPLE_SIZE)

spm.SentencePieceTrainer.Train(SPM_COMMAND)

In [ ]:
def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format(MODEL_PREFIX))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

In [ ]:
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token

In [ ]:
bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

In [ ]:
ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
bert_vocab = ctrl_symbols + bert_vocab

In [ ]:
bert_vocab += ["[UNUSED_{}]".format(i) for i in range(VOC_SIZE - len(bert_vocab))]
print(len(bert_vocab))

In [ ]:
VOC_FNAME = "vocab.txt" #@param {type:"string"}

with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

# Now Start Pretraining Step with BERT scripts

In [ ]:
! pip install -U tokenizers
import tokenizers


In [ ]:
bwpt = tokenizers.BertWordPieceTokenizer(
    vocab=None,
    unk_token='[UNK]',
    sep_token='[SEP]',
    cls_token='[CLS]',
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=True,
    lowercase=True,
    wordpieces_prefix='##'
)

In [ ]:
bwpt.train(
    files=["../input/51k-pretrain-dataset/51k.txt"],
    vocab_size=30000,
    min_frequency=3,
    limit_alphabet=1000,
    special_tokens=['[PAD]', '[UNK]', '[CLS]', '[MASK]', '[SEP]']
)

In [ ]:
bwpt.save("/kaggle/working/tokenized.txt")

In [ ]:
!cd bert

In [ ]:
!ls

In [ ]:
!python ./bert/create_pretraining_data.py \
    --input_file=../input/51k-pretrain-dataset/51k.txt \
    --output_file=/kaggle/working/tf_examples.tfrecord \
    --vocab_file=./vocab.txt\
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=42 \
    --dupe_factor=5

In [ ]:
!python ../input/bertsrc/run_pretraining.py \
    --input_file=gs://pretrain-bucket/tf_examples.tfrecord \
    --output_dir=gs://pretrain-bucket/51k_pretrain_model/ \
    --do_train=True \
    --do_eval=True \
    --bert_config_file=/kaggle/input/bert-base-uncased/config.json \
    --train_batch_size=32 \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --num_train_steps=20 \
    --num_warmup_steps=10 \
    --learning_rate=2e-5 \
    --use_tpu=True \
    --tpu_name=$TPU_NAME